### SynthPop Explanation
This notebook summarizes the way SynthPop works.

First some notes about the structure of synthpop. 
It has a few dependencies: 
- `census`, a python library that is a wrapper for the U.S. Census Bureau API
- `pandas` a python library used to create and work with dataframes (like tables)
- `numpy`
- `os`which allows you to set up a development environment (not actually entirely necessary)

SynthPop itself is actually a few separate scripts that each handle different aspects of the synthesizing process:  

If you are using ACS estimates via the Census API then you use the following sets of SynthPop tools 
- `synthesizer.py` which relies on a 'recipe' that is the output of:
    - `starter.py` which relies on data queried and formatted from the Census API by:
        - `census_helpers.py` which relies on the Census API python wrapper provided by: 
            - `census.py`
            
You can also use SynthPop with any other data source (i.e. not ACS data). To do this you use: 
- `zone_synthesizer.py`which a set of functions that accepts marginals and sample files from a CSV and produces a synthesized population. 

Here are slightly more detailed descriptions about each element of SynthPop: 

The Synth pop library `census_helpers.py` relies on `census` and is a set of funcitons to assist with downloading and processing census data for a given geography. It allows you to select geography and columns of interest and to download data at the block or tract level (or both).  

`synthesizer.py` uses a 'recipe' which is the output of the `starter.py` script.  

`starter.py` uses `census_helpers.py` to generate and return: 
    - household marginals
    - person marginals
    - household joint distribution
    - person joint distribution
    - tract to PUMA map (a disctionary showing the relationship between tracts and PUMAs)
It returns them as a 'recipe' i.e. not easily accessible as individual files. 

### Using SynthPop:
**Step 1:**
install SynthPop and dependencies:
1. Install Python (Anaconda recommended), and depdendencies: 
    `census`
    `numexpr`
    `numpy`
    `pandas`
    `scipy`
    `us`
(Everything except cenus and us are included with Anaconda.)

2. Download the source code from here on GitHub. Install SynthPop by running python setup.py install in the synthpop directory.

**Step 2:**  
1. Load in libraries

2. Set API key. (If you don't already have you can get one [here](https://api.census.gov/data/key_signup.html))

In [1]:
%load_ext autoreload
%autoreload 2
from synthpop.recipes.starter2 import Starter
from synthpop.synthesizer import synthesize_all, enable_logging, synthesize
import os
import pandas as pd
import csv
enable_logging()

# setting API Key
os.environ["CENSUS"] = "d95e144b39e17f929287714b0b8ba9768cecdc9f"

### Synthesizer for a list of tracts
To speed up the process of generating synthetic populations I created a script that would synthesize populations by census tract for a list of tracts (saving a csv file for each tract) and then ran this script in parallel on different chunks of tracts for Mecklenburg County. 
To speed up processing I saved this snippet of code as a .py file and ran it directly from the command line.

with `python nameoffile.py`

In [ ]:
# %load_ext autoreload
# %autoreload 2
from synthpop.recipes.starter2 import Starter
from synthpop.synthesizer import synthesize_all, enable_logging
import os
import pandas as pd
import csv
enable_logging()

# setting API Key
os.environ["CENSUS"] = "d95e144b39e17f929287714b0b8ba9768cecdc9f"

# funciton that save a csv for the results for each tract it is given
def synthesize_save(tract):
    print("tract number %s" %tract)
    starter = Starter(os.environ["CENSUS"], "NC", "Mecklenburg County",tract)
    print("tract number %s" %tract)
#   using synthesize_all2 (see below) so that the person-level data has a geoid
    all_households, all_persons, fit_quality = synthesize_all2(starter)
    all_households.to_csv('data_outputs/%s_households.csv'%tract)
    all_persons.to_csv('data_outputs/%s_persons.csv'%tract)
    with open('data_outputs/%s_fit.csv'%tract, 'w') as f:
        for key in fit_quality.keys():
            f.write("%s,%s\n"%(key,fit_quality[key]))

    #function that iterates over multiple tracts
def synthesize_tracts_save(tracts):
    for tract in tracts:
        synthesize_save(tract)

#groupings of tracts for mecklenburg county:
tracts1 = ["006209","005521","005511","005614","005611","005200","002003","003105","980300","003007","003013","005815"]
tracts2 = ["000900","000100","001000","000300","001100","001200","001300","005715","005828","005836","003806","005834","003805"]
tracts3 = ["003203","003108","002702","001609","001504","005843","005616","001505","005841","001507","001603","006007","005512","006303","001921","005716"]
tracts4 = ["001922","002906","005830","000400","005832","000500","003204","000600","980200","000700","005912","005909","005906","006008","006215","000800","005508","005509","005518","005522","005714","006006","005826","005847","005842","005908","006211","006210","006304","006404","006403","005618","005917","006212","005915","001923","004303","005517","001917","005524","005520","005835","005617","005604","005838","005845","006104","004000","004100","004200","005615","005612","006407","006108","005825","006103","003802","006405","003902","005717","005848","006009"]
tracts5=["005605","005610","005706","005911","005709","006214","005712","005846","005713","004302","005000","005823","004500","006107","006203","004600","005609","001912","004700","005833","003006","004800","001509","005812","005100","001801","005817","005301","001608","003109","005827","001915","006109","001702","001605","001802"]
tracts6 = ["001607","001606","005305","001701","001920","001910","003017","005840","005910","005510","005513","002002","003016","003102","006213","003103","005837","005514","005619","980100","005307","005306","001508","005613","005916","003808","005918","001510","005914","005308"]
tracts7 = ["005523","001919","001914","001918","002100","001916","002200","002300","002400","005621","005620","002905","002500","005831","003201","003300","003400","005829","003500","005839","003106","003018","005913","003807","005907","003903","006005","002600","002800","003011","003600"]
tracts4rep = ["005842","005908","006211","006210","006304","006404","006403","005618","005917","006212","005915","001923","004303","005517","001917","005524","005520","005835","005617","005604","005838","005845","006104","004000","004100","004200","005615","005612","006407","006108","005825","006103","003802","006405","003902","005717","005848","006009"]
tracts8= ["003700","004305","004304","006010","006208","002004","006406","003008","005711","003012","002903","005811","002904","005710","001911","003015","004400","005401","006204","001400","006302","005824","002701","005404","005403","006106","005519","005844","005516","004900","005515","006105","005816"]
tracts4rep2 = ["005604","005838","005845","006104","004000","004100","004200","005615","005612","006407","006108","005825","006103","003802","006405","003902","005717","005848","006009"]

synthesize_tracts_save(tracts4rep2)

###  New synthesizer that preserves geoid for person dataframe:


In [31]:
import logging
import sys
from collections import namedtuple

import numpy as np
import pandas as pd
from scipy.stats import chisquare

logger = logging.getLogger("synthpop")
FitQuality = namedtuple(
    'FitQuality',
    ('people_chisq', 'people_p'))
BlockGroupID = namedtuple(
    'BlockGroupID', ('state', 'county', 'tract', 'block_group'))


def enable_logging():
    handler = logging.StreamHandler(stream=sys.stdout)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)



def synthesize_all2(recipe, num_geogs=None, indexes=None,
                   marginal_zero_sub=.01, jd_zero_sub=.001):
    """
    Returns
    -------
    households, people : pandas.DataFrame
    fit_quality : dict of FitQuality
        Keys are geographic IDs, values are namedtuples with attributes
        ``.household_chisq``, ``household_p``, ``people_chisq``,
        and ``people_p``.

    """
    print("Synthesizing at geog level: '{}' (number of geographies is {})"
          .format(recipe.get_geography_name(), recipe.get_num_geographies()))

    if indexes is None:
        indexes = recipe.get_available_geography_ids()

    hh_list = []
    people_list = []
    cnt = 0
    fit_quality = {}
    hh_index_start = 0

    # TODO will parallelization work here?
    for geog_id in indexes:
        print("Synthesizing geog id:\n", geog_id)

        h_marg = recipe.get_household_marginal_for_geography(geog_id)
        logger.debug("Household marginal")
        logger.debug(h_marg)

        p_marg = recipe.get_person_marginal_for_geography(geog_id)
        logger.debug("Person marginal")
        logger.debug(p_marg)

        h_pums, h_jd = recipe.\
            get_household_joint_dist_for_geography(geog_id)
        logger.debug("Household joint distribution")
        logger.debug(h_jd)

        p_pums, p_jd = recipe.get_person_joint_dist_for_geography(geog_id)
        logger.debug("Person joint distribution")
        logger.debug(p_jd)

        households, people, people_chisq, people_p = \
            synthesize(
                h_marg, p_marg, h_jd, p_jd, h_pums, p_pums,
                marginal_zero_sub=marginal_zero_sub, jd_zero_sub=jd_zero_sub,
                hh_index_start=hh_index_start)

        # Append location identifiers to the synthesized households
        for geog_cat in geog_id.keys():
            households[geog_cat] = geog_id[geog_cat]
            
        # DB added : Append location identifiers to the synthesized people
        for geog_cat in geog_id.keys():
            people[geog_cat] = geog_id[geog_cat]
        

        hh_list.append(households)
        people_list.append(people)
        key = BlockGroupID(
            geog_id['state'], geog_id['county'], geog_id['tract'],
            geog_id['block group'])
        fit_quality[key] = FitQuality(people_chisq, people_p)

        cnt += 1
        if len(households) > 0:
            hh_index_start = households.index.values[-1] + 1

        if num_geogs is not None and cnt >= num_geogs:
            break

    # TODO might want to write this to disk as we go?
    all_households = pd.concat(hh_list)
    all_persons = pd.concat(people_list)

    return (all_households, all_persons, fit_quality)

### Synthesis of a whole county with pre-set variables:
The following code takes a long time (multiple hours) to execute but will run the synthesize for block groups in a whole county. 

Note: the county name must be 'Name County' ie 'Kings County' otherwise it will not work. 

In [ ]:
# TAKES FOREVER TO RUN PLEASE SKIP
def synthesize_counties(counties):
    for county in counties:
        starter = Starter(os.environ["CENSUS"], "NC", county)
        synthesize_all(starter)
%time hh = synthesize_counties(["Mecklenburg County"]) 

### Synthesis by tract

In [28]:
# making a recipe for just one tract
starter_tract = Starter(os.environ["CENSUS"], "NC", "Mecklenburg County","005706")

In [29]:
# synthesizing housholds based on that recipe
all_households_005706, all_persons_005706, fit_quality_005706 = synthesize_all(starter_tract)

Synthesizing at geog level: 'block_group' (number of geographies is 4)
Synthesizing geog id:
 state              37
county            119
tract          005706
block group         1
dtype: object
Household marginal
cat_name         cat_value      
hh_age_of_head   gt35-lt65          421
                 gt65               150
                 lt35                80
hh_cars          none                14
                 one                188
                 two or more        446
hh_children      no                 405
                 yes                246
hh_income        gt100-lt150        129
                 gt150               34
                 gt30-lt60          241
                 gt60-lt100         124
                 lt30               123
hh_race_of_head  asian               17
                 black              121
                 other               48
                 white              465
hh_size          four or more       159
                 one            

Person joint distribution
                                        cat_id  frequency
hispanic person_age   person_sex race                    
no       19 and under female     asian       0       70.0
                                 black       1      278.0
                                 other       2       63.0
                                 white       3      771.0
                      male       asian       4       59.0
                                 black       5      290.0
                                 other       6       45.0
                                 white       7      751.0
         20 to 35     female     asian       8       67.0
                                 black       9      195.0
                                 other      10       20.0
                                 white      11      604.0
                      male       asian      12       74.0
                                 black      13      185.0
                                 other      14

Running ipu


/Users/darebrawley/anaconda3/lib/python3.6/site-packages/synthpop/ipu/ipu.py:190: RuntimeWarning: divide by zero encountered in double_scalars
  adj = constraint / float((column * weights).sum())


Time to run ipu: 53.990s
IPU weights:
count    3.687000e+03
mean     1.789821e-01
std      3.304002e-01
min      4.273394e-07
25%      4.897452e-03
50%      5.317941e-02
75%      2.202046e-01
max      4.524072e+00
dtype: float64
Fit quality:
1.1696776516833296
Number of iterations:
27
Drawing 650 households
Synthesizing geog id:
 state              37
county            119
tract          005706
block group         2
dtype: object
Household marginal
cat_name         cat_value      
hh_age_of_head   gt35-lt65          378
                 gt65               132
                 lt35                84
hh_cars          none                13
                 one                171
                 two or more        407
hh_children      no                 375
                 yes                219
hh_income        gt100-lt150         51
                 gt150               15
                 gt30-lt60          124
                 gt60-lt100         167
                 lt30             

...

Time to run ipu: 61.453s
IPU weights:
count     3.687000e+03
mean      2.209778e-01
std       7.992793e-01
min      3.601089e-126
25%       4.156984e-06
50%       1.100297e-04
75%       1.566895e-01
max       1.850452e+01
dtype: float64
Fit quality:
3.2030048600645014
Number of iterations:
31
Drawing 673 households
Synthesizing geog id:
 state              37
county            119
tract          005706
block group         4
dtype: object
Household marginal
cat_name         cat_value      
hh_age_of_head   gt35-lt65          293
                 gt65               228
                 lt35               100
hh_cars          none                14
                 one                179
                 two or more        427
hh_children      no                 506
                 yes                115
hh_income        gt100-lt150         71
                 gt150               41
                 gt30-lt60          170
                 gt60-lt100         196
                 lt30     

Person joint distribution
                                        cat_id  frequency
hispanic person_age   person_sex race                    
no       19 and under female     asian       0       70.0
                                 black       1      278.0
                                 other       2       63.0
                                 white       3      771.0
                      male       asian       4       59.0
                                 black       5      290.0
                                 other       6       45.0
                                 white       7      751.0
         20 to 35     female     asian       8       67.0
                                 black       9      195.0
                                 other      10       20.0
                                 white      11      604.0
                      male       asian      12       74.0
                                 black      13      185.0
                                 other      14

Running ipu


/Users/darebrawley/anaconda3/lib/python3.6/site-packages/synthpop/ipu/ipu.py:190: RuntimeWarning: divide by zero encountered in double_scalars
  adj = constraint / float((column * weights).sum())


Time to run ipu: 437.228s
IPU weights:
count    3.687000e+03
mean     1.933344e-01
std      4.484030e-01
min      3.711018e-11
25%      4.032434e-06
50%      7.556055e-05
75%      1.988441e-01
max      7.685979e+00
dtype: float64
Fit quality:
4.872272957062106
Number of iterations:
234
Drawing 620 households


### Test synthesis for just one block

In [2]:
starter = Starter(os.environ["CENSUS"], "NC", "Mecklenburg County")

In [3]:
ind = pd.Series(["37", "119", "005706", "4"], index=["state", "county", "tract", "block group"])


In [17]:
synthesize_all(starter, indexes=[ind])

Synthesizing at geog level: 'block_group' (number of geographies is 555)
Synthesizing geog id:
 state              37
county            119
tract          005706
block group         4
dtype: object
Household marginal
cat_name         cat_value      
hh_age_of_head   gt35-lt65          293
                 gt65               228
                 lt35               100
hh_cars          none                14
                 one                179
                 two or more        427
hh_children      no                 506
                 yes                115
hh_income        gt100-lt150         71
                 gt150               41
                 gt30-lt60          170
                 gt60-lt100         196
                 lt30               143
hh_race_of_head  asian               16
                 black               48
                 other               21
                 white              536
hh_size          four or more       113
                 one          

Person joint distribution
                                        cat_id  frequency
hispanic person_age   person_sex race                    
no       19 and under female     asian       0       70.0
                                 black       1      278.0
                                 other       2       63.0
                                 white       3      771.0
                      male       asian       4       59.0
                                 black       5      290.0
                                 other       6       45.0
                                 white       7      751.0
         20 to 35     female     asian       8       67.0
                                 black       9      195.0
                                 other      10       20.0
                                 white      11      604.0
                      male       asian      12       74.0
                                 black      13      185.0
                                 other      14

Running ipu


/Users/darebrawley/anaconda3/lib/python3.6/site-packages/synthpop/ipu/ipu.py:190: RuntimeWarning: divide by zero encountered in double_scalars
  adj = constraint / float((column * weights).sum())


KeyboardInterrupt: 

In [18]:
all_households, all_persons, fit_quality = synthesize_all(starter, indexes=[ind])

Synthesizing at geog level: 'block_group' (number of geographies is 555)
Synthesizing geog id:
 state              37
county            119
tract          005706
block group         4
dtype: object
Household marginal
cat_name         cat_value      
hh_age_of_head   gt35-lt65          293
                 gt65               228
                 lt35               100
hh_cars          none                14
                 one                179
                 two or more        427
hh_children      no                 506
                 yes                115
hh_income        gt100-lt150         71
                 gt150               41
                 gt30-lt60          170
                 gt60-lt100         196
                 lt30               143
hh_race_of_head  asian               16
                 black               48
                 other               21
                 white              536
hh_size          four or more       113
                 one          

Person joint distribution
                                        cat_id  frequency
hispanic person_age   person_sex race                    
no       19 and under female     asian       0       70.0
                                 black       1      278.0
                                 other       2       63.0
                                 white       3      771.0
                      male       asian       4       59.0
                                 black       5      290.0
                                 other       6       45.0
                                 white       7      751.0
         20 to 35     female     asian       8       67.0
                                 black       9      195.0
                                 other      10       20.0
                                 white      11      604.0
                      male       asian      12       74.0
                                 black      13      185.0
                                 other      14

Running ipu


/Users/darebrawley/anaconda3/lib/python3.6/site-packages/synthpop/ipu/ipu.py:190: RuntimeWarning: divide by zero encountered in double_scalars
  adj = constraint / float((column * weights).sum())


Time to run ipu: 440.502s
IPU weights:
count    3.687000e+03
mean     1.933344e-01
std      4.484030e-01
min      3.711018e-11
25%      4.032434e-06
50%      7.556055e-05
75%      1.988441e-01
max      7.685979e+00
dtype: float64
Fit quality:
4.872272957062106
Number of iterations:
234
Drawing 620 households


In [22]:
fit_quality

{BlockGroupID(state='37', county='119', tract='005706', block_group='4'): FitQuality(people_chisq=338.05684048577194, people_p=8.187343855902998e-45)}